# <center> Data cleaning

## Summary
1. Remove duplicated records.
2. Replace negative item price and item count to median value.
3. Add missing months to train dataset.
4. Remove outliers
5. Fix shop names and category names.
6. Join test and train set. Set test set 34 month.
7. Drop ID column from test data set.
8. Save results to %store variables

## Import datasets

Read files from ETL notebook

In [10]:
%load_ext jupyternotify

%store -r item_cat
%store -r item
%store -r shops
%store -r sales_test
%store -r sales_train

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [11]:
__ipy

Helper ipython script loaded


In [12]:
__da

Basic Data Analysis tools was loaded


## Remove duplicated records

In [13]:
sales_train_cleaned = sales_train.copy()
duplicated_cols = sales_train.groupby(['date', 'item_id', 'shop_id'])['date_block_num'].count().reset_index()
duplicated_cols = duplicated_cols[duplicated_cols['date_block_num'] == 1].rename(columns={'date_block_num': 'count'})
duplicated_cols = sales_train.groupby(['date', 'item_id', 'shop_id'])['date_block_num'].count().reset_index()
duplicated_cols = duplicated_cols[duplicated_cols['date_block_num'] > 1]
duplicated_cols = duplicated_cols.reset_index(drop=True) 

sales_train_cleaned = sales_train_cleaned.drop_duplicates(['date', 'item_id', 'shop_id'])
sales_train.shape[0] - sales_train_cleaned.shape[0]

28

## Replace negative item price and item count to median value

In [14]:
median_2973_item_price = sales_train_cleaned['item_price'][sales_train_cleaned['item_id']  == 2934].median()
sales_train_cleaned['item_price'] = sales_train_cleaned["item_price"].replace(-1, value=median_2973_item_price)

assert sales_train_cleaned.loc[np.where(sales_train_cleaned['item_price'] < 0)[0], 'item_price'].shape[0] == 0

## Add missing months to train dataset & Remove outliers

In [21]:
%%notify

from itertools import product
import time
from tqdm import tqdm, trange

train = sales_train_cleaned

filled_train = []
cols = ['date_block_num','shop_id','item_id']

for i in trange(34):
    sales_month = train[train.date_block_num == i]
    all_key_pairs = list(product([i], sales_month.shop_id.unique(), sales_month.item_id.unique()))    
    filled_train.append(np.array(all_key_pairs, dtype='int16'))
   
filled_train_df = pd.DataFrame(np.vstack(filled_train), columns=cols)
filled_train_df['date_block_num'] = filled_train_df['date_block_num'].astype(np.int8)
filled_train_df['shop_id'] = filled_train_df['shop_id'].astype(np.int8)
filled_train_df['item_id'] = filled_train_df['item_id'].astype(np.int16)
filled_train_df.sort_values(cols,inplace=True)

group = train.groupby(['date_block_num','shop_id','item_id'])\
    .agg({'item_cnt_day': ['sum'], 'item_price': ['median']})

group.columns = ['item_cnt_month', 'item_price']
group.reset_index(inplace=True)

filled_train_df = pd.merge(filled_train_df, group, on=cols, how='left')
filled_train_df['item_cnt_month'] = (filled_train_df['item_cnt_month']
                                .fillna(0)
                                .clip(0,20) # NB clip target here
                                .astype(np.float32))

filled_train_df['item_price'] = (filled_train_df['item_price']
                                .fillna(0)
                                .astype(np.float32))

train = filled_train_df[(filled_train_df['item_price'] < 100000 ) & (filled_train_df['item_cnt_month'] < 1010)]

100%|██████████| 34/34 [00:06<00:00,  5.45it/s]


<IPython.core.display.Javascript object>

## Fix shop names and category names.

In [22]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

## Join test and train set. Set test set 34 month.

In [23]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

In [25]:
train_test = pd.concat([filled_train_df, test], ignore_index=True, sort=False, keys=cols)
train_test.fillna(0, inplace=True) 

0.5396749973297119

## Drop ID column from test data set

In [26]:
train_test = train_test.drop('ID', axis=1)

In [28]:
%store train
%store train_test

Stored 'train' (DataFrame)
Stored 'train_test' (DataFrame)


## Final results

In [115]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10913849 entries, 0 to 10913849
Data columns (total 5 columns):
date_block_num    int8
shop_id           int8
item_id           int16
item_cnt_month    float32
item_price        float32
dtypes: float32(2), int16(1), int8(2)
memory usage: 208.2 MB


In [116]:
train_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11128050 entries, 0 to 11128049
Data columns (total 5 columns):
date_block_num    int8
shop_id           int8
item_id           int16
item_cnt_month    float32
item_price        float32
dtypes: float32(2), int16(1), int8(2)
memory usage: 127.4 MB
